In [8]:
import csv as csv
import gradio as gradio
import joblib as joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import NuSVR

In [9]:
training_data = pd.read_csv("data/train_data.csv", index_col=0)
test_data = pd.read_csv("data/test_data.csv", index_col=0)

features = ["Plant_Production_GWh", "Population_k", "tmax"]
target = ["Max_Demand_GW"]

# MODEL TRAINING
# The input features
x_train = training_data[features]
y_train = training_data[target]

x_test = test_data[features]
y_test = test_data[target]

# Linear Model
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

# Polynomial Model
polynomial_model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
polynomial_model.fit(x_train, y_train)

# EXPORT MODELS AS JOBLIB
joblib.dump(linear_model, "linear_model.joblib")
joblib.dump(polynomial_model, "polynomial_model.joblib")

loaded_linear_model = joblib.load("linear_model.joblib")
loaded_polynomial_model = joblib.load("polynomial_model.joblib")

In [10]:
features = ["Population_k", "tavg"]
targets = ["Max_Demand_GW"]

# MODEL TRAINING
# Polynomial Model
polynomial_features = PolynomialFeatures(degree=2)

x_train_polynomial = polynomial_features.fit_transform(training_data[features])
nonlinear_model_v2 = LinearRegression()
nonlinear_model_v2.fit(x_train_polynomial, training_data[targets])

# Linear Model
features = ["Max_Demand_GW"]
targets = ["Plant_Production_GWh"]

linear_model_v2 = LinearRegression()
linear_model_v2.fit(training_data[features], training_data[targets])

# EXPORT MODELS AS JOBLIB
joblib.dump(nonlinear_model_v2, "nonlinear_model_v2.joblib")
joblib.dump(linear_model_v2, "linear_model_v2.joblib")

loaded_nonlinear_model = joblib.load("nonlinear_model_v2.joblib")
loaded_linear_model_v2 = joblib.load("linear_model_v2.joblib")

In [11]:
# MODEL TRAINING
# SVM
svm_features = ["tmax", "Population_k"]
svm_target = ["Max_Demand_GW"]

feature_ToTrain = training_data[svm_features]
target_ToTrain = training_data[svm_target]

feature_ToTrain = feature_ToTrain.to_numpy()
target_ToTrain = target_ToTrain.to_numpy().ravel()

nu_svr_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("nu_svr", NuSVR(nu=0.5589, C=0.01, degree=2, kernel="poly")),
    ]
)

nu_svr_pipeline.fit(feature_ToTrain, target_ToTrain)

# EXPORT MODEL AS JOBLIB
joblib.dump(nu_svr_pipeline, "svm.joblib")
loaded_svm_model = joblib.load("svm.joblib")

The TCN's architectre and weights were already saved in [nn_model.ipynb](./nn_model.ipynb) - **see** Using the Tensorflow Saved TCN

In [12]:
from tensorflow.keras.models import load_model

MODEL_PATH = "./models/tcn"
val_model = load_model(MODEL_PATH)

val_model.summary()

Model: "TCN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 12, 7)]      0           []                               
                                                                                                  
 Conv1D_1_0 (Conv1D)            (None, 12, 128)      3584        ['input_4[0][0]']                
                                                                                                  
 SpatialDropout1D_1_0 (SpatialD  (None, 12, 128)     0           ['Conv1D_1_0[0][0]']             
 ropout1D)                                                                                        
                                                                                                  
 Conv1D_2_0 (Conv1D)            (None, 12, 128)      65536       ['SpatialDropout1D_1_0[0][0]'] 